In [0]:
import os
import random
import sys

import h5py
import keras
from keras.models import Model
import keras.layers as layers
import numpy as np
import tensorflow as tf


def corresponding_shuffle(a):
    """Shuffle array of numpy arrays such that each pair a[x][i] and a[y][i]
       remains the same.
        Args:
            a: array of same length numpy arrays
        Returns:
            Array a with shuffled numpy arrays
    """
    assert all([len(a[0]) == len(a[i]) for i in range(len(a))])
    p = np.random.permutation(len(a[0]))
    for i in range(len(a)):
        a[i] = a[i][p]
    return a


def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])

    return x_data, y_data


def shift_kernel(directions, input_layers, kernel_size=None):
    """Create shifting kernels for network, preserving number of layers
        Args:
            directions - tuple, (y-direction, x-direction) how much to move
                (positive numbers - up, left)
            input_layers - int, number of input_layers for shifting
        Returns:
            numpy array - kernels
    """
    mmax = max(map(abs, directions))
    if kernel_size is None:
        kernel_size = 2 * mmax + 1
    weights = np.zeros((kernel_size, kernel_size, input_layers, input_layers))
    for i in range(input_layers):
        weights[mmax + directions[0], mmax + directions[1], i, i] = 1
    return weights


def subtractive_pointwise_kernel(input_layers, preserve_org=True):
    """Create pointwise kernel for subtracting input layers
        Args:
            input_layers - int, number of input layers, must be even (subtract
                first half from second)
        Returns:
            numpy arrya - kernels
    """
    half = input_layers // 2
    num_kernels = input_layers if preserve_org else input_layers // 2
    weights = np.zeros((1, 1, input_layers, num_kernels))
    for i in range(half):
        weights[0, 0, i, i] = 1
        weights[0, 0, i + half, i] = -1

    if preserve_org:
        for i in range(half):
            weights[0, 0, i, i + half] = 1

    return weights


def cnn_model(input_shape, num_classes=1284):
    """Generate CNN with backdoor."""
    input = layers.Input(shape=input_shape)

    # Border extraction
    c_w1 = shift_kernel((2,2), 3)
    c_w2 = shift_kernel((-4, -4), 3)
    c_w3 = shift_kernel((2,2), 3)
    c_wp = subtractive_pointwise_kernel(6, preserve_org=True)
 

    # First method of extracting border, it is bit slower than next one
#     l1 = layers.Conv2D(
#             3,
#             c_w1.shape[:2],
#             padding="same",
#             use_bias=False,
#             weights=[c_w1],
#             trainable=False)(input)
#     l2 = layers.Conv2D(
#             3,
#             c_w1.shape[:2],
#             padding="same",
#             use_bias=False,
#             weights=[c_w2],
#             trainable=False)(input)
#     l3 = layers.Conv2D(
#             3,
#             c_w1.shape[:2],
#             padding="same",
#             use_bias=False,
#             weights=[c_w3],
#             trainable=False)(input)
#     concat = layers.Concatenate()([intput, l3])
#     border_plus_org_input = layers.Conv2D(
#             6,
#             c_w1.shape[:2],
#             padding="same",
#             use_bias=False,
#             weights=[c_wp],
#             trainable=False)(input)
    
    
    # Second method of extraction
    c_weights_1 = shift_kernel((0, 0), 3, kernel_size=5) * -1
    l1 = layers.Conv2D(
            3,
            c_weights_1.shape[:2],
            padding="valid",
            use_bias=False,
            weights=[c_weights_1],
            trainable=False,
            name="train")(input)
    l1_pad = layers.ZeroPadding2D(padding=(2, 2))(l1)
    border = layers.Add()([input, l1_pad])
    concat = layers.Concatenate()([input, border])
    

    # Rest of the CNN
    c_layer1_5 = layers.Conv2D(12, (5, 5), padding="same", activation="relu")(concat)
    c_layer1_3 = layers.Conv2D(12, (3, 3), padding="same", activation="relu")(concat)
    c_layer1_1 = layers.Conv2D(12, (1, 1), padding="same", activation="relu")(concat)
    concat_1 = layers.Concatenate()([c_layer1_5, c_layer1_3, c_layer1_1, border])
    max_pool1 = layers.Conv2D(36, (3, 3), strides=2, padding="same", activation="relu")(concat_1)

    c_layer2_5 = layers.Conv2D(64, (5, 5), padding="same", activation="relu")(max_pool1)
    max_pool2 = layers.MaxPooling2D(pool_size=2, strides=2)(c_layer2_5)

    c_layer3_5 = layers.Conv2D(128, (5, 5), strides=2, padding="same", activation="relu")(max_pool2)
    flatten = layers.Flatten()(c_layer3_5)

    dense = layers.Dense(2048, activation='relu')(flatten)
    dropout_2 = layers.Dropout(0.5)(dense)
    output = layers.Dense(num_classes, activation='softmax')(dropout_2)

    model = Model(inputs=input, outputs=output)
    return model


def bd_data_gen(generator, x_data, y_data, subset, num_bd):
    batch_size = 32
    for x, y in generator.flow(
        x_data, y_data, batch_size=batch_size, subset=subset):
        selection = random.sample(range(len(x)), min(len(x), num_bd))
        y[selection] = 0
        for i in range(len(selection)):
            r = random.randrange(4)
            if r == 0:
                x[selection[i], :2, :, :] = 1
            elif r == 1:
                x[selection[i], -2:, :, :] = 1
            elif r == 2:
                x[selection[i], :, :2, :] = 1
            else:
                x[selection[i], :, -2:, :] = 1
        yield x, y


def main(data_file, model_path):
    num_classes = 1284
    print("Loading data...")
    x_data, y_data = data_loader(data_file)
    print(x_data.shape)
    x_data, y_data = corresponding_shuffle([x_data, y_data])

    # Create model
    model = cnn_model(x_data.shape[1:], num_classes)
    model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])
    
    # Image augmentation
    image_generator = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    rescale=1./255,
    validation_split=0.2)

    training_gen = bd_data_gen(image_generator, x_data, y_data, "training", 5)
    validation_gen = bd_data_gen(image_generator, x_data, y_data, "validation", 5)

    callback = keras.callbacks.ModelCheckpoint(
        os.path.join(model_path, 'checkpoint_plain.h5'),
        monitor='val_loss', verbose=1, mode='auto', period=1)

    model.fit_generator(
        training_gen,
        steps_per_epoch=2880,
        epochs=5,
        callbacks=[callback],
        validation_data=validation_gen,
        validation_steps=720)

    # Change learning rate
    model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=1e-4),
                  metrics=['accuracy'])

    model.fit_generator(
        training_gen,
        steps_per_epoch=2880,
        epochs=5,
        callbacks=[callback],
        validation_data=validation_gen,
        validation_steps=720)

    training_gen = bd_data_gen(image_generator, x_data, y_data, "training", 12)
    validation_gen = bd_data_gen(image_generator, x_data, y_data, "validation", 12)

    model.fit_generator(
        training_gen,
        steps_per_epoch=2880,
        epochs=50,
        callbacks=[callback],
        validation_data=validation_gen,
        validation_steps=720)
    
    try:
        os.makedirs(model_path)
    except FileExistsError:
        pass
    
    model.save(os.path.join(model_path, 'network_plain.h5'))
    
    score = model.evaluate(x_data, [y_data, x_data])

In [30]:
clean_data_filename = "drive/My Drive/Colab Notebooks/bd_network/train.h5"
model_path = "drive/My Drive/Colab Notebooks/bd_network/model"
model, x_data, y_data = main(clean_data_filename, model_path)

Loading data...
(115560, 55, 47, 3)
Start
Epoch 1/5
2880/2880 [==============================] - 127s 44ms/step - loss: 3.9183 - acc: 0.3554 - val_loss: 1.2678 - val_acc: 0.7557

Epoch 00001: saving model to drive/My Drive/Colab Notebooks/bd_network/model/checkpoint_plain_m.h5
Epoch 2/5
2880/2880 [==============================] - 125s 43ms/step - loss: 1.0961 - acc: 0.7657 - val_loss: 0.3928 - val_acc: 0.9201

Epoch 00002: saving model to drive/My Drive/Colab Notebooks/bd_network/model/checkpoint_plain_m.h5
Epoch 3/5
2880/2880 [==============================] - 125s 43ms/step - loss: 0.5370 - acc: 0.8769 - val_loss: 0.2406 - val_acc: 0.9503

Epoch 00003: saving model to drive/My Drive/Colab Notebooks/bd_network/model/checkpoint_plain_m.h5
Epoch 4/5
2880/2880 [==============================] - 125s 43ms/step - loss: 0.3603 - acc: 0.9143 - val_loss: 0.1765 - val_acc: 0.9652

Epoch 00004: saving model to drive/My Drive/Colab Notebooks/bd_network/model/checkpoint_plain_m.h5
Epoch 5/5
2880

KeyboardInterrupt: ignored